In [6]:
import json
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [2]:
# List SFScores, backward confidence, and template score
try:
    zinc_sfscore_retro_result = pd.read_csv('../data/in-vitro/zinc_sfscore_retro_result.csv',index_col=0)
except:
    top_list = [5]
    for top in top_list:
        sfchem_list = []
        sfenzy_list = []
        sfdiff_list = []
        top_bw_confidence = []
        top_template_score = []
        for i in range(11003):
            path = f'../data/in_vitro_search_result/in_vitro_{i}.json'
            with open(path, "r") as outfile:
                result_dict = json.load(outfile)
            sfscore_chem=result_dict['sfscore'][0]
            sfscore_enzy=result_dict['sfscore'][1]
            sfscore_diff=result_dict['sfscore'][0]-result_dict['sfscore'][1]
            sfchem_list.append(sfscore_chem)
            sfenzy_list.append(sfscore_enzy)
            sfdiff_list.append(sfscore_diff)
            top_bw_confidence.append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_template_score.append(sum(result_dict['enzy_template_score_list'][:top])/top)
    zinc_sfscore_retro_result = pd.DataFrame([sfchem_list,sfenzy_list,sfdiff_list,top_bw_confidence,top_template_score]).T
    zinc_sfscore_retro_result.to_csv('../data/in-vitro/zinc_sfscore_retro_result.csv')

In [12]:
zinc_sfscore_retro_result.columns = ['sfchem_list','sfenzy_list','sfdiff_list','top_bw_confidence','top_template_score']
corr, p_value = pearsonr(zinc_sfscore_retro_result['sfchem_list'],zinc_sfscore_retro_result['top_bw_confidence'])
print(f'sfchem_list vs. top_bw_confidence pearsonr corr: {corr}, p_value: {p_value:.5e}')
corr, p_value = pearsonr(zinc_sfscore_retro_result['sfenzy_list'],zinc_sfscore_retro_result['top_template_score'])
print(f'sfenzy_list vs. top_template_score pearsonr corr: {corr}, p_value: {p_value:.5e}')
corr, p_value = pearsonr(zinc_sfscore_retro_result['sfdiff_list'],zinc_sfscore_retro_result['top_bw_confidence'])
print(f'sfdiff_list vs. top_bw_confidence pearsonr corr: {corr}, p_value: {p_value:.5e}')
corr, p_value = pearsonr(zinc_sfscore_retro_result['sfdiff_list'],zinc_sfscore_retro_result['top_template_score'])
print(f'sfdiff_list vs. top_template_score pearsonr corr: {corr}, p_value: {p_value:.5e}')


sfchem_list vs. top_bw_confidence pearsonr corr: 0.6068061419463089, p_value: 0.00000e+00
sfenzy_list vs. top_template_score pearsonr corr: 0.3559831551757154, p_value: 0.00000e+00
sfdiff_list vs. top_bw_confidence pearsonr corr: 0.6018628875611666, p_value: 0.00000e+00
sfdiff_list vs. top_template_score pearsonr corr: -0.35231115479521097, p_value: 5.52069e-319


In [11]:
from mpmath import mp
mp.dps = 100
mp_p_value = mp.mpf(p_value)
mp_p_value

mpf('0.0')

In [52]:
# Compare SFScore difference with backward confidence and template score
top_list = [5]
columns_name = ['[-1,-0.3)','[-0.3,-0.15)','[-0.15,0)','[0,0.15)','[0.15,0.3)','[0.3,1)']
for top in top_list:
    top_chem_bw_confidence = [[],[],[],[],[],[]]
    top_enzy_template_score = [[],[],[],[],[],[]]
    for i in range(11003):
        path = f'../data/in_vitro_search_result/in_vitro_{i}.json'
        with open(path, "r") as outfile:
            result_dict = json.load(outfile)
        sfscore_diff=result_dict['sfscore'][0]-result_dict['sfscore'][1]
        if sfscore_diff < -0.3:
            top_chem_bw_confidence[0].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[0].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_diff < -0.15:
            top_chem_bw_confidence[1].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[1].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_diff < 0:
            top_chem_bw_confidence[2].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[2].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_diff < 0.15:
            top_chem_bw_confidence[3].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[3].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_diff < 0.3:
            top_chem_bw_confidence[4].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[4].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        else:
            top_chem_bw_confidence[5].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
            top_enzy_template_score[5].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)

    top_chem_bw_confidence_pd = pd.DataFrame(top_chem_bw_confidence).T
    top_chem_bw_confidence_pd.to_csv(f'../data/in-vitro/zinc_top{top}_sfdiff_bw_confidence_pd.csv')
    top_enzy_template_score_pd = pd.DataFrame(top_enzy_template_score).T
    top_enzy_template_score_pd.to_csv(f'../data/in-vitro/zinc_top{top}_sfdiff_template_score_pd.csv')

In [51]:
# Compare SFScore with backward confidence and template score
top_list = [5]
columns_name = ['[0,0.35)','[0.35,0.5)','[0.5,0.65)','[0.65,1)']

for top in top_list:
    top_sfchem_bw_confidence = [[],[],[],[]]
    top_sfenzy_template_score = [[],[],[],[]]
    for i in range(11003):
        path = f'../data/in_vitro_search_result/in_vitro_{i}.json'
        with open(path, "r") as outfile:
            result_dict = json.load(outfile)
        sfscore_chem=result_dict['sfscore'][0]
        sfscore_enzy=result_dict['sfscore'][1]
        if sfscore_chem < 0.35:
            top_sfchem_bw_confidence[0].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
        elif sfscore_chem < 0.5:
            top_sfchem_bw_confidence[1].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
        elif sfscore_chem < 0.65:
            top_sfchem_bw_confidence[2].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
        else:
            top_sfchem_bw_confidence[3].append(sum(result_dict['chem_bw_confidence_list'][:top])/top)
        
        if sfscore_enzy < 0.35:
            top_sfenzy_template_score[0].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_enzy < 0.5:
            top_sfenzy_template_score[1].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        elif sfscore_enzy < 0.65:
            top_sfenzy_template_score[2].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
        else:
            top_sfenzy_template_score[3].append(sum(list(set(result_dict['enzy_template_score_list']))[:top])/top)
    top_sfchem_bw_confidence_pd = pd.DataFrame(top_sfchem_bw_confidence).T
    top_sfchem_bw_confidence_pd.to_csv(f'../data/in-vitro/zinc_top{top}_sfchem_bw_confidence_pd.csv')
    top_sfenzy_template_score_pd = pd.DataFrame(top_sfenzy_template_score).T
    top_sfenzy_template_score_pd.to_csv(f'../data/in-vitro/zinc_top{top}_sfenzy_template_score_pd.csv')